# Attività di EDA su Wikipedia

## Descrizione del dataset

Il dataset offerto è composto da 4 colonne:

- **title**: indica il titolo dell'articolo
- **summary**: contiene l'introduzione dell'articolo
- **documents**: contiene l'articolo completo
- **categoria**: contiene la categoria associata all'articolo

## Obiettivi

### 1. Attività EDA:
È necessario svolgere un'attività di EDA per analizzare e valutare statisticamente tutto il contenuto informativo offerto da Wikipedia. Il dataset fornito possiede le seguenti categorie:

- 'culture'
- 'economics'
- 'energy'
- 'engineering'
- 'finance'
- 'humanities'
- 'medicine'
- 'pets'
- 'politics'
- 'research'
- 'science'
- 'sports'
- 'technology'
- 'trade'
- 'transport'

Per ogni categoria, calcolare le seguenti informazioni:

1. Numero di articoli
2. Numero medio di parole utilizzate
3. Numero massimo di parole presenti nell'articolo più lungo
4. Numero minimo di parole presenti nell'articolo più corto
5. Per ogni categoria, individuare la nuvola di parole più rappresentativa


### 2. Sviluppo classificatore NLP articoli :

Dopo aver svolto l'analisi richiesta, addestrare e testare un classificatore testuale capace di classificare gli articoli (secondo le categorie presenti nel dataset) che saranno in futuro inseriti.


## 2. Implementazione Classificatore Testuale :


In questa fase del nostro progetto, ci concentreremo su due obiettivi principali sullo sviluppare e valutare un modello di classificazione per articoli futuri

Il nostro obiettivo è creare un modello capace di classificare automaticamente nuovi articoli nelle categorie appropriate. <br>
Per fare ciò andremo ad eseguire le seguenti fasi implementative :

1. Preparazione dei Dati:
   - Divideremo il nostro dataset in set di addestramento e di test
   - Utilizziamo i dati vettorizzati per avere una rappresentazione numerica adatta al machine learning

2. Addestramento del Modello:
   - Sceglieremo un Logistic Regression come algoritmo di classificazione appropriato 
   - Addestreremo il modello sui dati di training

3. Valutazione del Modello:
   - Testeremo il modello sul set di dati di test
   - Valuteremo le performance usando metriche come accuratezza, precisione, recall e F1-score

Queste 4 fasi le effettueremo sia sul campo "summary" che sul campo "documents" a fine di verificare quale dei due fornisce le performance migliori in riferimento alle categorie che saranno la nostra variabile target.

**Risultato Atteso**

Al termine di questo processo, avremo un modello di classificazione che sarà in grado di:
- Analizzare il contenuto di nuovi articoli
- Assegnare a questi articoli la categoria più probabile basandosi sulle caratteristiche apprese dal nostro corpus

Questo strumento sarà prezioso per automatizzare la classificazione di futuri articoli, rendendo più efficiente il processo di categorizzazione e organizzazione dei contenuti.

Nelle prossime celle di codice, implementeremo questi passaggi utilizzando le funzionalità di machine learning di PySpark.

Iniziamo il progetto andandoci a scaricare il dataset di lavoro :

In [0]:
!wget https://proai-datasets.s3.eu-west-3.amazonaws.com/wikipedia.csv

--2024-08-10 12:38:06--  https://proai-datasets.s3.eu-west-3.amazonaws.com/wikipedia.csv
Resolving proai-datasets.s3.eu-west-3.amazonaws.com (proai-datasets.s3.eu-west-3.amazonaws.com)... 3.5.225.182, 52.95.154.36
Connecting to proai-datasets.s3.eu-west-3.amazonaws.com (proai-datasets.s3.eu-west-3.amazonaws.com)|3.5.225.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1003477941 (957M) [text/csv]
Saving to: ‘wikipedia.csv.1’

wikipedia.csv.1     100%[===================>] 956.99M  19.0MB/s    in 67s     

2024-08-10 12:39:14 (14.4 MB/s) - ‘wikipedia.csv.1’ saved [1003477941/1003477941]



Ci predisponiamo il Dataframe spark di lavoro seguendo il seguente codice che va a leggere da un cluster AWS S3 il file csv e lo importa in un dataframe spark.

N.b:
Nel codice che segue, eseguiremo un campionamento stratificato del nostro dataset. Questo approccio è necessario  poiché in fase di sviluppo stiamo lavorando in un ambiente Databricks Community, pertanto avremo accesso a risorse computazionali limitate.

**Processo di campionamento**

1. Estrarremo un campione bilanciato del dataset originale.
2. Utilizzeremo un metodo di campionamento stratificato basato sulla colonna "categoria".
3. Questo significa che la distribuzione delle categorie nel nostro campione sarà proporzionalmente la stessa del dataset originale.

Questo approccio ci permetterà di lavorare con un dataset più piccolo e gestibile, mantenendo allo stesso tempo la rappresentatività dei nostri dati originali.

In [0]:

import pandas as pd
from sklearn.model_selection import train_test_split

dataset = pd.read_csv('/databricks/driver/wikipedia.csv')
categoria_col = 'categoria'  

# Riduciamo la size per agevolare il running nella versione community di databricks e lo facciamo con campionamento stratificato (in modo che si mantiene bilanciato)
dataset_sample, _ = train_test_split(dataset, test_size=0.01, stratify=dataset[categoria_col], random_state=42)

spark_df_ = spark.createDataFrame(dataset_sample)
spark_df_ = spark_df_.drop("Unnamed: 0")
spark_df_.write.mode("overwrite").saveAsTable("wikipedia")




Mostriamo il contenuto del dataframe spark di lavoro:

In [0]:
spark_df_.show()

+--------------------+--------------------+--------------------+-----------+
|               title|             summary|           documents|  categoria|
+--------------------+--------------------+--------------------+-----------+
|         york street|york street, also...|york street, also...|       pets|
|             sedbury|sedbury is a vill...|sedbury is a vill...|engineering|
|      john johnston |john johnston (17...|john johnston (17...|      trade|
|crossboundary energy|crossboundary ene...|crossboundary ene...|     energy|
|panki thermal pow...|panki thermal pow...|panki thermal pow...|     energy|
|            rappbode|the rappbode is a...|the rappbode is a...|engineering|
|1955 u.s. nationa...|first-seeded dori...|first-seeded dori...|     sports|
|      pauline hanson|pauline lee hanso...|pauline lee hanso...|   politics|
|boronia railway s...|boronia railway s...|boronia railway s...|  transport|
|zetes power stations|the zonguldak ere...|the zonguldak ere...|     energy|

### a. Modello per campo "Summary"



Implementeremo adesso una pipeline di Data Cleaning utilizzando SparkNLP. Il processo include:

1. Normalizzazione del testo
2. Lemmatizzazione
3. Rimozione delle stopwords

**Applicazione e Risultati**

- DataFrame "df_cleaned_summary": Versione pulita della colonna "summary" su nuova colonna "cleaned_text"
 

In [0]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType


# Per estrarre il testo pulito in una nuova colonna:
from pyspark.sql.functions import concat_ws
import re


# Definiamo la pipeline di Spark NLP che pulisce il testo che agisce su "summary".

# Crea un DocumentAssembler
# Questo componente prende il testo grezzo e lo converte in un documento annotato con metadati di riferimento
# setInputCol("text"): specifica la colonna di input contenente il testo grezzo
# setOutputCol("document"): specifica la colonna di output per il documento annotato
document_assembler = DocumentAssembler().setInputCol("summary").setOutputCol("document")

# Crea un Tokenizer
# Questo componente divide il testo in singole parole o token
# setInputCols(["document"]): specifica la colonna di input (il documento annotato)
# setOutputCol("token"): specifica la colonna di output per i token
tokenizer = Tokenizer().setInputCols(["document"]).setOutputCol("token")

# Crea un Normalizer
# Questo componente normalizza il testo, ad esempio convertendolo in minuscolo
# setInputCols(["token"]): specifica la colonna di input (i token)
# setOutputCol("normalized"): specifica la colonna di output per i token normalizzati
# setLowercase(True): imposta la conversione in minuscolo
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setCleanupPatterns(["[^\w\s]"]) \
    .setLowercase(True) \
    .setOutputCol("normalized") \
    .setLowercase(True)

# Crea un LemmatizerModel
# Questo componente riduce le parole alla loro forma base (lemma)
# pretrained(): carica un modello pre-addestrato per la lemmatizzazione
# setInputCols(["normalized"]): specifica la colonna di input (i token normalizzati)
# setOutputCol("lemma"): specifica la colonna di output per i lemmi
lemmatizer = LemmatizerModel.pretrained().setInputCols(["normalized"]).setOutputCol("lemma")

# Crea uno StopWordsCleaner
# Questo componente rimuove le parole comuni (stopwords) che spesso non portano significato
# pretrained(): carica una lista pre-definita di stopwords
# setInputCols(["lemma"]): specifica la colonna di input (i lemmi)
# setOutputCol("cleaned"): specifica la colonna di output per le parole pulite
stopwords_cleaner = StopWordsCleaner.pretrained().setInputCols(["lemma"]).setOutputCol("cleaned")

# Crea il pipeline
nlp_pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer, 
    normalizer, 
    lemmatizer, 
    stopwords_cleaner
])

# Funzione per applicare la pipeline e pulire il testo
def clean_text_spark_nlp(df, input_col="summary", output_col="cleaned_text"):
    # Fit della pipeline 
    fitted_pipeline = nlp_pipeline.fit(df)
    
    # Applica il pipeline al DataFrame
    result = fitted_pipeline.transform(df)

    # Estraiamo il testo pulito in una nuova colonna
    result_with_clean_text = result.withColumn(output_col, concat_ws(" ", "cleaned.result"))

    # Restituiamo il dataframe con la selezione delle colonne originali e la nuova colonna di testo pulito
    return result_with_clean_text.select(df.columns + [output_col])

# Applichiamo la funzione al tuo DataFrame
df_cleaned_summary = clean_text_spark_nlp(spark_df_)

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
stopwords_en download started this may take some time.
Approximate size to download 2.9 KB
[OK!]


Mostriamo adesso la struttura del df "df_cleaned_summary" con la colonna aggiunta:

In [0]:
df_cleaned_summary.show()

+--------------------+--------------------+--------------------+-----------+--------------------+
|               title|             summary|           documents|  categoria|        cleaned_text|
+--------------------+--------------------+--------------------+-----------+--------------------+
|         york street|york street, also...|york street, also...|       pets|york street jakem...|
|             sedbury|sedbury is a vill...|sedbury is a vill...|engineering|sedbury village f...|
|      john johnston |john johnston (17...|john johnston (17...|      trade|john johnston 176...|
|crossboundary energy|crossboundary ene...|crossboundary ene...|     energy|crossboundary ene...|
|panki thermal pow...|panki thermal pow...|panki thermal pow...|     energy|panki thermal pow...|
|            rappbode|the rappbode is a...|the rappbode is a...|engineering|rappbode righthan...|
|1955 u.s. nationa...|first-seeded dori...|first-seeded dori...|     sports|firstseeded doris...|
|      pauline hanso

Quello che ora dobbiamo fare sarà quello di definire la vettorizzazione tramite HashingTF più efficace del CountVectorizer in quanto mappa direttamente le parole in indici del vettore delle feature usando una funzione di hash. Questo riduce il tempo e la memoria necessari per costruire un vocabolario esplicito, come invece avviene con l'ausilio del  CountVectorizer :

In [0]:
from pyspark.ml.feature import HashingTF
from pyspark.sql.functions import udf, sum as spark_sum, split
from pyspark.sql.types import IntegerType

# Tokenizziamo la colonna "cleaned_text" 
df_cleaned_summary = df_cleaned_summary.withColumn("words", split(df_cleaned_summary.cleaned_text, "\\s+"))


# Definiamo HashingTF sulla colonna "words"
hashingTF = HashingTF(inputCol="words", outputCol="word_vector") 

# Effettuo la Caching dei dati per ridurre i tempi di lettura
df_cleaned_summary.cache()

# Applico la trasformazione
df_cleaned_summary_cont_vect = hashingTF.transform(df_cleaned_summary)



Andiamo a vedere il tipo di colonna che mi ha generato nel dataframe che abbiamo denominato "word_vector": 

In [0]:
df_cleaned_summary_cont_vect.printSchema()

root
 |-- title: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- documents: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- cleaned_text: string (nullable = false)
 |-- words: array (nullable = false)
 |    |-- element: string (containsNull = false)
 |-- word_vector: vector (nullable = true)



Abbiamo ottenuto una nuova colonna chiamata "word_vector". Questa colonna contiene vettori sparsi che rappresentano il conteggio delle parole per ogni documento. Ecco come procederemo per analizzare questi dati:

1. Comprensione della Colonna "word_vector"
- La colonna "word_vector" è di tipo "vector".
- Ogni elemento di questo vettore rappresenta una parola unica nel vocabolario.
- I valori diversi da zero in questo vettore indicano la presenza e la frequenza di una parola nel documento.

Procedo a crearmi il Classificatore ed addestrarlo per la colonna "summary" e mostriamo le relative performance. <br> 

Iniziamo a rendere codificabile la variabile target "categoria":

In [0]:
from pyspark.sql.functions import col, create_map, lit
from pyspark.sql.types import IntegerType
from itertools import chain

def custom_category_indexer(df, input_col, output_col):
    # Ottieni tutte le categorie uniche
    categories = df.select(input_col).distinct().rdd.flatMap(lambda x: x).collect()
    
    # Crea un dizionario di mapping categoria -> indice
    category_dict = {cat: idx for idx, cat in enumerate(categories)}
from pyspark.sql.functions import col, monotonically_increasing_id, row_number
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

def optimized_category_indexer(df, input_col, output_col):
    # Crea un DataFrame con categorie uniche e indici
    category_df = df.select(input_col).distinct()
    
    # Usa row_number per assegnare indici univoci
    window = Window.orderBy(monotonically_increasing_id())
    category_df = category_df.withColumn(output_col, row_number().over(window) - 1)
    
    # Esegui un join per assegnare gli indici al DataFrame originale
    df_indexed = df.join(category_df, on=input_col, how="left")
    
    return df_indexed.withColumn(output_col, col(output_col).cast(IntegerType()))

# Uso della funzione
df_cleaned_summary_cont_vect = optimized_category_indexer(df_cleaned_summary_cont_vect, "categoria", "categoriaIndex")

# Dividiamo il dataset in training e test
train, test = df_cleaned_summary_cont_vect.randomSplit([0.7, 0.3], seed=42)

# Cache dei dati per migliorare le prestazioni
train.cache()
test.cache()


Procediamo adesso all'implementazione ed addestramento del modello:

In [0]:
from pyspark.ml.classification import LogisticRegression

# Modello 
lr = LogisticRegression(featuresCol="word_vector", labelCol="categoriaIndex", maxIter=3)
model_documents = lr.fit(train)

# Predizioni
predictions_documents = model_documents.transform(train)


# Valutiamo il modello con la colonna corretta per la classificazione
evaluator = MulticlassClassificationEvaluator(labelCol="categoriaIndex", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_documents)
print(f"Accuracy: {accuracy}")

# Mostriamo alcune predizioni, usando le colonne corrette
predictions_documents.select("categoria", "categoriaIndex", "prediction", "probability").show(10)

Accuracy: 0.8830082319255548
+----------+----------------------+----------+--------------------+
| categoria|categoriaIndex_summary|prediction|         probability|
+----------+----------------------+----------+--------------------+
|technology|                   8.0|       8.0|[7.58220915235418...|
|    sports|                  14.0|      14.0|[2.02391659606794...|
|    sports|                  14.0|      14.0|[1.32120731570598...|
|    sports|                  14.0|      14.0|[8.40768403813396...|
|    sports|                  14.0|      14.0|[1.01821115752587...|
|    sports|                  14.0|      14.0|[9.87376050369236...|
|    sports|                  14.0|      14.0|[1.20523668366701...|
|    sports|                  14.0|      14.0|[4.89522599126926...|
|    sports|                  14.0|      14.0|[7.60023971594721...|
|    sports|                  14.0|      14.0|[2.48079553034763...|
+----------+----------------------+----------+--------------------+
only showing top 10